<a href="https://colab.research.google.com/github/SathishMalaVish/Data-Driven-Stock-Analysis/blob/main/Final_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit
!pip install mysql-connector-python
!pip install pyngrok
!npm install localtunnel
!pip install ngrok
!pip install mysql
!pip install requests pandas pymysql streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 642.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.0/34.0 MB 13.7 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦
added 22 packages in 2s
⠦
⠦3 packages are looking for funding
⠦  run `npm fund` for details
⠦Collecting ngrok
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.4/91.4 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mysqlclient: filename=mysqlclient-2.2.7-cp311-cp311-linux_x86_64.whl size=127529 sha256=65273d7dc0db180935861a3561b1d6a83d32c2a670006058687d2f325ef2d8a3
  Stored in dir

In [ ]:
# Converting all Yaml file into CSV file, combine and save the CSV files, symbol wise csv file save

import os
import yaml
import csv
import pandas as pd
from collections import defaultdict
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Update paths to Google Drive or Colab directory
folder_path = "/content/drive/My Drive/Data-Driven Stock Analysis"
output_dir = "/content/drive/My Drive/Assignment_CSV/symbol_CSV"

# Ensure output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def extract_and_transform_data(folder_path, output_dir):
    symbol_data = defaultdict(list)

    # Debugging: List files
    print("Scanning folder:", folder_path)

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.yaml'):
                file_path = os.path.join(root, file)
                print(f"Processing file: {file_path}")

                try:
                    with open(file_path, 'r', encoding='utf-8') as yaml_file:
                        yaml_content = yaml.safe_load(yaml_file)

                        if yaml_content and isinstance(yaml_content, list):
                            for entry in yaml_content:
                                if isinstance(entry, dict) and 'Ticker' in entry:
                                    ticker = entry['Ticker']
                                    flattened_data = {
                                        'ticker': ticker,
                                        'date': entry.get('date', ''),
                                        'close': entry.get('close', ''),
                                        'high': entry.get('high', ''),
                                        'low': entry.get('low', ''),
                                        'open': entry.get('open', ''),
                                        'volume': entry.get('volume', ''),
                                        'month': entry.get('month', '')
                                    }
                                    symbol_data[ticker].append(flattened_data)
                                else:
                                    print(f"Invalid entry in {file_path}")
                        else:
                            print(f"Invalid YAML format: {file_path}")

                except yaml.YAMLError as e:
                    print(f"Error reading YAML: {file_path}, {e}")
                except Exception as e:
                    print(f"Unexpected error: {file_path}, {e}")

    # Write to CSV
    for symbol, entries in symbol_data.items():
        output_csv = os.path.join(output_dir, f"{symbol}.csv")
        with open(output_csv, 'w', newline='', encoding='utf-8') as csv_file:
            fieldnames = ['ticker', 'date', 'close', 'high', 'low', 'open', 'volume', 'month']
            csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
            csv_writer.writeheader()

            for entry in entries:
                if isinstance(entry, dict):
                    csv_writer.writerow(entry)
                else:
                    print(f"Unexpected data format: {entry}")

        print(f"CSV created: {output_csv}")

extract_and_transform_data(folder_path, output_dir)

save_path = "/content/drive/My Drive/Assignment_CSV/combined_data.csv"  # Path to save the combined CSV

# Load all CSV files
def load_all_csvs_and_save(output_dir, save_path):
    data_frames = []
    print("📂 Scanning CSV files in:", output_dir)

    for file in os.listdir(output_dir):
        if file.endswith('.csv'):
            file_path = os.path.join(output_dir, file)
            print(f"📄 Loading file: {file_path}")
            try:
                df = pd.read_csv(file_path)
                df['ticker'] = file.replace('.csv', '')
                df['date'] = pd.to_datetime(df['date'], errors='coerce')  # Convert date column to datetime
                data_frames.append(df)
            except Exception as e:
                print(f"❌ Error loading {file_path}: {e}")

    if data_frames:
        # Combine all dataframes into one
        combined_df = pd.concat(data_frames, ignore_index=True)
        combined_df = combined_df.sort_values(by=['ticker', 'date'])

        # Save the combined DataFrame to a new CSV file
        combined_df.to_csv(save_path, index=False)
        print(f"✅ Combined data saved to: {save_path}")
    else:
        print("❌ No CSV files found or loaded.")

# Run the function to load, combine, and save the CSV files
load_all_csvs_and_save(output_dir, save_path)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Scanning folder: /content/drive/My Drive/Data-Driven Stock Analysis
Processing file: /content/drive/My Drive/Data-Driven Stock Analysis/2024-11/2024-11-11_05-30-00.yaml
Processing file: /content/drive/My Drive/Data-Driven Stock Analysis/2024-11/2024-11-21_05-30-00.yaml
Processing file: /content/drive/My Drive/Data-Driven Stock Analysis/2024-11/2024-11-18_05-30-00.yaml
Processing file: /content/drive/My Drive/Data-Driven Stock Analysis/2024-11/2024-11-22_05-30-00.yaml
Processing file: /content/drive/My Drive/Data-Driven Stock Analysis/2024-11/2024-11-04_05-30-00.yaml
Processing file: /content/drive/My Drive/Data-Driven Stock Analysis/2024-11/2024-11-19_05-30-00.yaml
Processing file: /content/drive/My Drive/Data-Driven Stock Analysis/2024-11/2024-11-08_05-30-00.yaml
Processing file: /content/drive/My Drive/Data-Driven Stock Analysis/2024-11/2024-11-06_05-30-00.

In [9]:
# Stock_Data Imported into TiDB

import pandas as pd
import pymysql
from sqlalchemy import create_engine

# 🔹 Step 1: Mount Google Drive (Ensure CSV file is accessible)
from google.colab import drive
drive.mount('/content/drive')

# 🔹 Step 2: Set Paths
output_dir = "/content/drive/My Drive/Assignment_CSV/symbol_CSV"  # Folder with CSV files
save_path = "/content/drive/My Drive/Assignment_CSV/combined_data.csv"  # Combined CSV path
certificate_path = "/content/drive/MyDrive/Assignment_CSV/isrgrootx1.pem"  # SSL certificate path

# 🔹 Step 4: Connect to TiDB
tidb_host = "gateway01.ap-southeast-1.prod.aws.tidbcloud.com"
tidb_port = 4000
tidb_user = "3nitUUKsZ9vMwCU.root"
tidb_password = "rXPvkXcel0MlnGKI"
tidb_database = "Project_2"
tidb_table = "stock_data"

# Create connection with SSL
connection = pymysql.connect(
    host=tidb_host,
    port=tidb_port,
    user=tidb_user,
    password=tidb_password,
    ssl={'ca': certificate_path}  # Secure connection
)

cursor = connection.cursor()

# 🔹 Step 5: Create Database (If Not Exists)
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {tidb_database};")
print(f"✅ Database '{tidb_database}' checked/created successfully!")

# Use the new database
cursor.execute(f"USE {tidb_database};")

# 🔹 Step 6: Create Table (If Not Exists)
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {tidb_table} (
    id INT AUTO_INCREMENT PRIMARY KEY,
    ticker VARCHAR(512) NOT NULL,
    date DATETIME,
    close DOUBLE,
    high DOUBLE,
    low DOUBLE,
    open DOUBLE,
    volume BIGINT,
    month VARCHAR(512)
);
"""
cursor.execute(create_table_query)
print(f"✅ Table '{tidb_table}' checked/created successfully!")

# Commit the changes and close cursor
connection.commit()
cursor.close()
connection.close()

# 🔹 Step 7: Load Data from CSV
df = pd.read_csv(save_path)

# 🔹 Step 8: Insert Data into TiDB using SQLAlchemy
if df is not None:
    # Reconnect using SQLAlchemy for bulk insert
    engine = create_engine(
        f"mysql+pymysql://{tidb_user}:{tidb_password}@{tidb_host}:{tidb_port}/{tidb_database}",
        connect_args={"ssl": {"ca": certificate_path}}
    )

    df.to_sql(tidb_table, engine, if_exists="append", index=False)
    print(f"✅ Data successfully imported into '{tidb_table}' in TiDB!")
else:
    print("❌ No data to import!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Database 'Project_2' checked/created successfully!
✅ Table 'stock_data' checked/created successfully!
✅ Data successfully imported into 'stock_data' in TiDB!


In [18]:
#Sector File Import in TIDB

import pandas as pd
from sqlalchemy import create_engine

tidb_host = "gateway01.ap-southeast-1.prod.aws.tidbcloud.com"
tidb_port = 4000
tidb_user = "3nitUUKsZ9vMwCU.root"
tidb_password = "rXPvkXcel0MlnGKI"
tidb_database = "Project_2"
tidb_table = "sector_data"
certificate_path = "/content/drive/MyDrive/Assignment_CSV/isrgrootx1.pem"

# Create SQLAlchemy engine with SSL
engine = create_engine(
    f"mysql+pymysql://{tidb_user}:{tidb_password}@{tidb_host}:{tidb_port}/{tidb_database}",
    connect_args={"ssl": {"ca": certificate_path}}
)

# Specify the path to your CSV file
csv_file_path = "/content/drive/MyDrive/Assignment_CSV/sector/sector_data.csv"

try:
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file_path)

    # Upload the DataFrame to TiDB
    df.to_sql(tidb_table, engine, if_exists='replace', index=False)

    print(f"Data from '{csv_file_path}' successfully uploaded to TiDB table '{tidb_table}'")

except FileNotFoundError:
    print(f"Error: CSV file not found at {csv_file_path}")
except Exception as e:
    print(f"An error occurred: {e}")


Data from '/content/drive/MyDrive/Assignment_CSV/sector/sector_data.csv' successfully uploaded to TiDB table 'sector_data'


In [19]:
# # Data Cleaning in Stock Data

import pandas as pd
from sqlalchemy import create_engine, text
import pymysql

# Database credentials
tidb_host = "gateway01.ap-southeast-1.prod.aws.tidbcloud.com"
tidb_port = 4000
tidb_user = "3nitUUKsZ9vMwCU.root"
tidb_password = "rXPvkXcel0MlnGKI"
tidb_database = "Project_2"
certificate_path = "/content/drive/MyDrive/Assignment_CSV/isrgrootx1.pem"
tidb_table = "stock_data"

# Create engine with SSL for secure connection
engine = create_engine(
    f"mysql+pymysql://{tidb_user}:{tidb_password}@{tidb_host}:{tidb_port}/{tidb_database}",
    connect_args={"ssl": {"ca": certificate_path}}
)

try:
    # Read data from TiDB into DataFrame
    query = f"SELECT * FROM {tidb_table}"
    df = pd.read_sql_query(query, engine)

    # Clean the data
    df.drop_duplicates(subset=['ticker', 'date'], inplace=True)
    df.fillna(0, inplace=True)
    df['volume'] = pd.to_numeric(df['volume'], errors='coerce').fillna(0).astype(int)

    # Convert 'date' column to datetime
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df['day_of_week'] = df['date'].dt.dayofweek  # Monday=0, Sunday=6

    # Remove weekends (Saturday=5, Sunday=6)
    df = df[~df['day_of_week'].isin([5, 6])]
    df.drop('day_of_week', axis=1, inplace=True)

    # Split 'date' into separate date and time columns
    df['date_only'] = df['date'].dt.date
    df['time_only'] = df['date'].dt.time
    df.drop('date', axis=1, inplace=True)

    # Upload cleaned data back to TiDB
    df.to_sql(tidb_table, engine, if_exists='replace', index=False)
    print(f"Cleaned and updated data in TiDB table '{tidb_table}'")

    # Rename columns in TiDB
    with engine.connect() as connection:
        connection.execute(text("ALTER TABLE stock_data CHANGE COLUMN date_only date DATE;"))
        connection.execute(text("ALTER TABLE sector_data CHANGE COLUMN symbol ticker VARCHAR(512);")
    )
    print("Columns renamed successfully")

except Exception as e:
    print(f"An error occurred: {e}")


Cleaned and updated data in TiDB table 'stock_data'
Columns renamed successfully


In [20]:
# Data Cleanning in Sector_data CSV

from sqlalchemy import create_engine, text
import pymysql

# Database credentials (replace with your actual credentials)
tidb_host = "gateway01.ap-southeast-1.prod.aws.tidbcloud.com"
tidb_port = 4000
tidb_user = "3nitUUKsZ9vMwCU.root"
tidb_password = "rXPvkXcel0MlnGKI"
tidb_database = "Project_2"
certificate_path = "/content/drive/MyDrive/Assignment_CSV/isrgrootx1.pem"
tidb_table = "sector_data"

# Create engine with SSL
engine = create_engine(
    f"mysql+pymysql://{tidb_user}:{tidb_password}@{tidb_host}:{tidb_port}/{tidb_database}",
    connect_args={"ssl": {"ca": certificate_path}},
    echo=True
)

try:
    with engine.begin() as connection:
        # Update ticker column values
        connection.execute(text(f"UPDATE {tidb_table} SET ticker = REPLACE(ticker, 'ADANIGREEN', 'ADANIENT');"))
        connection.execute(text(f"UPDATE {tidb_table} SET ticker = REPLACE(ticker, 'AIRTEL', 'BHARTIARTL');"))
        connection.execute(text(f"UPDATE {tidb_table} SET ticker = REPLACE(ticker, 'TATACONSUMER', 'TATACONSUM');"))
        print("Ticker column updated successfully.")

        # Insert new row
        connection.execute(text(
            f"INSERT INTO {tidb_table} (COMPANY, sector, ticker) VALUES ('Britannia Industries Ltd', 'FOOD & TOBACCO', 'BRITANNIA');"
        ))
        print("New row added successfully.")

except Exception as e:
    print(f"An error occurred: {e}")


2025-02-25 01:26:41,813 INFO sqlalchemy.engine.Engine SELECT DATABASE()


INFO:sqlalchemy.engine.Engine:SELECT DATABASE()


2025-02-25 01:26:41,814 INFO sqlalchemy.engine.Engine [raw sql] {}


INFO:sqlalchemy.engine.Engine:[raw sql] {}


2025-02-25 01:26:41,926 INFO sqlalchemy.engine.Engine SELECT @@sql_mode


...

INFO:sqlalchemy.engine.Engine:INSERT INTO sector_data (COMPANY, sector, ticker) VALUES ('Britannia Industries Ltd', 'FOOD & TOBACCO', 'BRITANNIA');


2025-02-25 01:26:42,269 INFO sqlalchemy.engine.Engine [generated in 0.00127s] {}


INFO:sqlalchemy.engine.Engine:[generated in 0.00127s] {}


New row added successfully.
2025-02-25 01:26:42,324 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import streamlit as st
import pandas as pd
import plotly.express as px
from sqlalchemy import create_engine
import pymysql

# Database credentials (Consider using environment variables for security)
tidb_host = "gateway01.ap-southeast-1.prod.aws.tidbcloud.com"
tidb_port = 4000
tidb_user = "3nitUUKsZ9vMwCU.root"
tidb_password = "rXPvkXcel0MlnGKI"
tidb_database = "Project_2"
stock_table = "stock_data"
sector_table = "sector_data"
certificate_path = "/content/drive/MyDrive/Assignment_CSV/isrgrootx1.pem"


# Create SQLAlchemy engine with SSL
engine = create_engine(
    f"mysql+pymysql://{tidb_user}:{tidb_password}@{tidb_host}:{tidb_port}/{tidb_database}",
    connect_args={"ssl": {"ca": certificate_path}}
)

@st.cache_data
def load_data():
    """Load stock and sector data from TiDB"""
    stock_query = f"SELECT * FROM {stock_table}"
    sector_query = f"SELECT * FROM {sector_table}"

    stock_df = pd.read_sql_query(stock_query, engine)
    sector_df = pd.read_sql_query(sector_query, engine)

    # Ensure 'date' column is in datetime format
    stock_df['date'] = pd.to_datetime(stock_df['date'])

    # Remove duplicates from sector data
    sector_df = sector_df.drop_duplicates(subset=['ticker'])

    # Merge stock and sector data
    merged_df = pd.merge(stock_df, sector_df, on='ticker', how='left')

    return merged_df

# Load Data
df = load_data()

def set_sidebar_color(bg_color="#00AEC1", text_color="#FFFFFF", selectbox_text_color="#000000"):
    st.markdown(
        f"""
        <style>
        /* Sidebar background and general text styling */
        [data-testid="stSidebar"] {{
            background-color: {bg_color} !important;
        }}

        /* General sidebar text - white and bold */
        [data-testid="stSidebar"] * {{
            color: {text_color} !important;
            font-weight: normal !important;
        }}

        /* Selectbox label - white and bold */
        [data-testid="stSidebar"] .stSelectbox label {{
            color: {text_color} !important;
            font-weight: normal !important;
        }}

        /* Selectbox dropdown and selected text - black */
        [data-testid="stSidebar"] .stSelectbox div[data-baseweb="select"] * {{
            color: {selectbox_text_color} !important;
        }}
        </style>
        """,
        unsafe_allow_html=True
    )

# Sidebar Navigation
set_sidebar_color(bg_color="#00AEC1", text_color="#FFFFFF", selectbox_text_color="#000000")   ##E15F99
st.sidebar.markdown(f'<iframe src="https://lottie.host/embed/234ce960-07ce-4fa8-8375-35afadf4297b/or8NrvgTaM.lottie" width="100" height="100" style="border:none;"></iframe>', unsafe_allow_html=True)  # Use st.markdown to embed the Lottie animation as an iframe
st.sidebar.title("📈 Stock Market Analysis")
st.sidebar.subheader("Select an Analysis")


menu = st.sidebar.radio("Navigation", ["🏠 Home", "📊 Analysis"])

# Home Page
if menu == "🏠 Home":
    st.title("🏠 Welcome to Stock Market Analysis Dashboard")
    st.write("""
    This dashboard provides interactive stock market insights,
    including volatility analysis, cumulative returns, sector performance,
    stock correlations, and monthly gainers & losers.

    Use the **sidebar navigation** to explore different analyses.
    """)
    st.markdown(f'<div style="display: flex; justify-content: flex-end;"> <iframe src="https://lottie.host/embed/1e25b53d-e914-4fbd-9650-5373dd6b8d7a/VSa0qQMh3S.lottie" width="250" height="250" style="border:none;"></iframe>', unsafe_allow_html=True)


# Analysis Section (Only appears when "Analysis" is selected)
elif menu == "📊 Analysis":
    st.sidebar.subheader("Select an Analysis Type")

    # Radio buttons for different analysis options
    option = st.sidebar.radio("",
                              ["Volatility Analysis", "Cumulative Return", "Sector Performance", "Stock Correlation", "Monthly Gainers & Losers"])

    # Volatility Analysis
    if option == "Volatility Analysis":
        st.title("📊 Top 10 Most Volatile Stocks")
        st.write("""
        Volatility represents the degree of variation in a stock's price.
        A higher volatility indicates more risk and potential reward.
        """)

        # Calculate daily return
        df['daily_return'] = df.groupby('ticker')['close'].pct_change()

        # Calculate volatility (standard deviation of daily returns)
        volatility = df.groupby('ticker')['daily_return'].std()

        # Convert volatility Series to DataFrame
        top_10_volatile = volatility.nlargest(10).reset_index()
        top_10_volatile.columns = ['ticker', 'volatility']  # Rename columns for clarity

        # Create a bar chart with distinct colors
        fig = px.bar(
            data_frame=top_10_volatile,  # Ensure DataFrame is explicitly passed
            x='ticker',
            y='volatility',
            labels={'ticker': 'Stock Ticker', 'volatility': 'Volatility'},
            title='Top 10 Most Volatile Stocks',
            color='ticker',  # Assign a unique color to each stock
            color_discrete_sequence=px.colors.qualitative.Bold  # Use a diverse color palette
        )

        # Display the Plotly chart in Streamlit
        st.plotly_chart(fig)

    # Cumulative Return Analysis
    elif option == "Cumulative Return":
        st.title("📈 Cumulative Return of Top 5 Performing Stocks")

        st.write("""
        Cumulative return represents the total gain or loss of a stock over time,
        assuming all gains are reinvested.
        This analysis highlights the top-performing stocks.
        """)

        # Ensure 'date' column is in datetime format
        df['date'] = pd.to_datetime(df['date'])

        # Sort data by date
        df = df.sort_values(by=['ticker', 'date'])

        # Compute daily returns
        df['daily_return'] = df.groupby('ticker')['close'].pct_change()
        df = df.dropna(subset=['daily_return'])

        # Compute cumulative return using .transform()
        df['cumulative_return'] = df.groupby('ticker')['daily_return'].transform(lambda x: (1 + x).cumprod()) - 1

        # Get top 5 stocks based on the latest cumulative return
        top_5_stocks = df.groupby('ticker')['cumulative_return'].last().nlargest(5).index

        # Filter for top 5 stocks
        top_5_df = df[df['ticker'].isin(top_5_stocks)]

        # Plot cumulative return for top 5 stocks
        fig = px.line(top_5_df, x='date', y='cumulative_return', color='ticker',
                      labels={'date': 'Date', 'cumulative_return': 'Cumulative Return'},
                      title='📌 Cumulative Return for Top 5 Stocks')

        st.plotly_chart(fig)


    # Sector Performance
    elif option == "Sector Performance":
        st.title("📊 Average Yearly Return by Sector")

        st.write("""
        This analysis calculates the average yearly return of different sectors,
        helping to identify the best and worst-performing industries.
        """)

        stock_query = f"SELECT * FROM {stock_table}"
        stock_df = pd.read_sql_query(stock_query, engine)

        # Read sector data
        sector_query = f"SELECT * FROM {sector_table}"
        sector_df = pd.read_sql_query(sector_query, engine)
        # Ensure 'date' is in datetime format
        stock_df['date'] = pd.to_datetime(stock_df['date'])

        # Remove duplicates from sector data (if any)
        sector_df = sector_df.drop_duplicates(subset=['ticker'])

        # Map sector information to stock_df
        stock_df['sector'] = stock_df['ticker'].map(sector_df.set_index('ticker')['sector'])

        # Sort data by date in ascending order before aggregation
        stock_df = stock_df.sort_values(by=['date', 'ticker'], ascending=True)

        # Extract year
        stock_df['year'] = stock_df['date'].dt.year


        # Calculate yearly returns and include sector
        yearly_returns = stock_df.groupby(['ticker', 'year', 'sector']).agg(
            first_close=('close', 'first'),
            last_close=('close', 'last')
        ).reset_index()

        # Compute yearly return percentage
        yearly_returns['yearly_return'] = (yearly_returns['last_close'] / yearly_returns['first_close']) - 1

        # Group by sector and calculate average yearly return
        sector_performance = yearly_returns.groupby('sector')['yearly_return'].mean().reset_index()

        # Rename column for clarity
        sector_performance.columns = ['sector', 'avg_yearly_return']

        # Create the Plotly bar chart
        fig = px.bar(
            data_frame=sector_performance,
            x='sector',
            y='avg_yearly_return',
            labels={'sector': 'Sector', 'avg_yearly_return': 'Average Yearly Return'},
            title="📌 Sector-wise Average Yearly Return",
            color='sector',  # Assign a unique color to each sector
            color_discrete_sequence=px.colors.qualitative.Dark24  # Use a diverse color palette
        )

        # Display the Plotly chart in Streamlit
        st.plotly_chart(fig)

    # Stock Correlation
    elif option == "Stock Correlation":
        st.title("📊 Stock Price Correlation Heatmap")

        pivot_df = df.pivot(index='date', columns='ticker', values='close')
        correlation_matrix = pivot_df.corr()

        fig = px.imshow(correlation_matrix, labels=dict(x="Stock Ticker", y="Stock Ticker", color="Correlation"),
                        x=correlation_matrix.columns, y=correlation_matrix.columns, color_continuous_scale='RdBu',
                        title="Stock Price Correlation Heatmap")

        fig.update_layout(
            width=1000,  # Increase width
            height=800,  # Increase height
            xaxis_title="Stock Ticker",
            yaxis_title="Stock Ticker",
            title={'text': "Stock Price Correlation Heatmap", 'y': 0.95, 'x': 0.5, 'xanchor': 'center', 'yanchor': 'top'}
        )
        st.plotly_chart(fig)

    # Monthly Gainers & Losers
    elif option == "Monthly Gainers & Losers":
        st.title("📊 Monthly Gainers & Losers")

        df['month'] = df['date'].dt.month
        df['year'] = df['date'].dt.year

        selected_year = st.sidebar.selectbox("📆 Select Year", df['year'].unique())
        selected_month = st.sidebar.selectbox("📆 Select Month", df['month'].unique())

        st.write(f"""
        Below are the top 5 gaining and losing stocks for **{selected_year}-{selected_month:02d}**
        based on their **monthly return** percentage.
        """)

        monthly_data = df[(df['year'] == selected_year) & (df['month'] == selected_month)]

        if not monthly_data.empty:
            monthly_data['monthly_return'] = monthly_data.groupby('ticker')['close'].pct_change()
            monthly_data['monthly_return'] = monthly_data['monthly_return'].fillna(0)

            last_monthly_returns = monthly_data.groupby('ticker')['monthly_return'].last()
            top_5_gainers = last_monthly_returns.nlargest(5)
            top_5_losers = last_monthly_returns.nsmallest(5)

            col1, col2 = st.columns(2)

            with col1:
                fig_gainers = px.bar(x=top_5_gainers.index, y=top_5_gainers.values,
                                    labels={'x': 'Stock Ticker', 'y': 'Monthly Return'},
                                    title=f' 🟢 Top 5 Gainers - {selected_year}-{selected_month:02}',
                                    color_discrete_sequence=['green']  # Setting bars to green
                                    )
                st.plotly_chart(fig_gainers)

            with col2:
                fig_losers = px.bar(x=top_5_losers.index, y=top_5_losers.values,
                                    labels={'x': 'Stock Ticker', 'y': 'Monthly Return'},
                                    title=f' 🔴 Top 5 Losers - {selected_year}-{selected_month:02}',
                                    color_discrete_sequence=['red'])  # Setting bars to red
                st.plotly_chart(fig_losers)

        else:
            st.warning(f"⚠ No data found for {selected_year}-{selected_month:02}")

# Footer
st.sidebar.markdown("---")
st.sidebar.info(
    "📊 **Stock Analysis Dashboard**  \n"
    "👩‍💻 Developed by **SATHISH**  \n"
    "📡 Data Source: TiDB  \n"
    "🔗 [GitHub Repository](https://github.com/SathishMalaVish/Data-Driven-Stock-Analysis)")



2025-02-25 01:18:42.690 No runtime found, using MemoryCacheStorageManager
2025-02-25 01:18:42.696 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-25 01:18:42.696 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-25 01:18:42.698 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-25 01:18:42.699 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-25 01:18:42.700 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-25 01:18:42.701 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-25 01:18:42.701 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-25 01:18:42.702 Thread 'MainThread':

DeltaGenerator(_root_container=1, _parent=DeltaGenerator())

In [14]:

# Set your ngrok auth token
from pyngrok import ngrok
ngrok.set_auth_token("2qZXTI70wbnYK9o4Ldi7bDiVMwX_6N2MUv8REzFnRsePg6zhh")


# Start ngrok tunnel
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at: {public_url}")

# Run the Streamlit app
!streamlit run work.py &>/dev/null&

Streamlit app is live at: NgrokTunnel: "https://6ca0-35-236-184-48.ngrok-free.app" -> "http://localhost:8501"
